In [69]:
import numpy as np
#from scipy.signal import butter, filtfilt
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
#import seaborn as sns
#sns.set_context("notebook", font_scale=1.3,
#                rc={'lines.linewidth': 1.5, 'lines.markersize': 14, 'axes.titlesize': 'x-large'})
#matplotlib.rc('legend', numpoints=1, fontsize=14)
#import glob
import sys, os
sys.path.insert(1, r'./Functions')
# IPython widgets:
from IPython.display import display
import ipywidgets
from ipywidgets import FloatProgress, interactive
import seaborn as sns
from scipy.signal import detrend

In [70]:
#!pip install version_information
%load_ext version_information
%version_information numpy, scipy, pandas, matplotlib, seaborn

The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information


Software versions
Python 3.8.8 64bit [MSC v.1916 64 bit (AMD64)]
IPython 7.22.0
OS Windows 10 10.0.22621 SP0
numpy 1.20.1
scipy 1.6.2
pandas 1.2.4
matplotlib 3.3.4
seaborn 0.11.1
Thu Apr 20 12:13:29 2023 Hora oficial do Brasil

In [71]:
# GitHub URL:
#path2 = 'C:\Users\ferna\Downloads\IC- dados\Dados'
#local directory:
path2 = r'C:\\Users\\ferna\\Downloads\\IC- dados\\Dados'
#path2 = '/home/rnwatanabe/copdata'
fname = os.path.join(path2, 'PDSinfo.txt')
PDSinfo = pd.read_csv(fname, sep='\t', header=0, index_col=None, engine='c', encoding='utf-8')
print(fname)
print("Information of %s subjects loaded (%s rows, %s columns)."
      %(len(pd.unique(PDSinfo.Subject)), PDSinfo.shape[0], PDSinfo.shape[1]))

C:\\Users\\ferna\\Downloads\\IC- dados\\Dados\PDSinfo.txt
Information of 49 subjects loaded (588 rows, 29 columns).


In [72]:
a=PDSinfo[PDSinfo['Subject']==44].index.tolist()
e=PDSinfo[PDSinfo['Subject']==44].index.tolist()[2]
ei = PDSinfo[PDSinfo['Subject']==44]

In [73]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
fs = 100 # Hz
import pickle 
#file = open('important', 'wb')

COPX = np.zeros((49*12, 60*fs-30))
COPZ = np.zeros((49*12, 60*fs-30))

COPXreal = np.zeros((49*12, 60*fs-30))
COPYreal = np.zeros((49*12, 60*fs-30))
COPZreal = np.zeros((49*12, 60*fs-30))

d1x = np.zeros((49*12, 60*fs-30))
d2x = np.zeros((49*12, 60*fs-30))

d1z = np.zeros((49*12, 60*fs-30))
d2z = np.zeros((49*12, 60*fs-30))

matrizangx = np.zeros((49*12, 60*fs-30))
matrizangz = np.zeros((49*12, 60*fs-30))

alpha = np.zeros((49*12, 60*fs-30))


ASISX = np.empty((49*12, 60*fs-30))
ASISY = np.empty((49*12, 60*fs-30))
ASISZ = np.empty((49*12, 60*fs-30))

d = np.zeros((49*12, 60*fs-30))


angulox = np.zeros((49*12, 60*fs-30))
anguloz= np.zeros((49*12, 60*fs-30))

matrizangx = np.zeros((49*12, 60*fs-30))
matrizangz = np.zeros((49*12, 60*fs-30))


subject = np.zeros((0,1))
j=0


trial = 0
for s in range (1,50):
    
    #a = PDSinfo[PDSinfo['Subject']==s].index.tolist()[2]
    
    ei = PDSinfo[PDSinfo['Subject']==s].index.tolist()
    
    
    for i in range(0,len(ei)):
        a = ei[i]
        fname_grf = os.path.join(path2, PDSinfo.Trial[a] + 'grf' + '.txt') 
        fname_mkr = os.path.join(path2, PDSinfo.Trial[a] + 'mkr' + '.txt')
        grf = pd.read_csv(fname_grf, delimiter='\t', header=0, engine='c')
        mkr = pd.read_csv(fname_mkr, delimiter='\t', header=0, engine='c')
        mkr = mkr.replace(np.nan,0)
        grf = grf.replace(np.nan,0)
        
        ASISX[trial,:] =(mkr['R.ASIS_X'].values[30:])
        ASISY[trial,:] =(mkr['R.ASIS_Y'].values[30:])
        ASISZ[trial,:] =(mkr['R.ASIS_Z'].values[30:])
        
        
        
        mass=PDSinfo['Mass']
        d[trial,:] = np.sqrt(ASISX[trial,:]**2+ASISY[trial,:]**2+ASISZ[trial,:]**2)
        angulox[trial,:] = np.arcsin(ASISX[trial,:]/d[trial,:])
        anguloz[trial,:] = np.arcsin(ASISZ[trial,:]/d[trial,:])
        sinx = np.sin(angulox[trial,:])
        sinz = np.sin(anguloz[trial,:])
        cosx = np.cos(angulox[trial,:])
        cosz = np.cos(anguloz[trial,:])
        tanx = np.tan(angulox[trial,:])
        tanz = np.tan(anguloz[trial,:])
        matrizangx[trial,:] = angulox[trial,:]
        matrizangz[trial,:] = anguloz[trial,:]


        Jb=(4/3)*(mass.values[s])*((d[trial,:])**2)
        t=grf['Time']
        dt=t[1]-t[0]


        d1x[trial,:] = np.gradient(matrizangx[trial,:],dt)
        d1z[trial,:] = np.gradient(matrizangz[trial,:],dt)

        d2x[trial,:] = np.gradient(d1x[trial,:],dt)
        d2z[trial,:] = np.gradient(d1z[trial,:],dt)

        COPX[trial,:] = d[trial,:]*sinx + Jb/(mass.values[s]*9.8)*(sinx*d1x[trial,:]**2-cosx*d2x[trial,:]) 
        COPZ[trial,:] = d[trial,:]*sinz + Jb/(mass.values[s]*9.8)*(sinz*d1z[trial,:]**2-cosz*d2z[trial,:])




        COPXreal[trial,:] = grf['COPNET_X'].values[30:]
        COPZreal[trial,:] = grf['COPNET_Z'].values[30:]

        subject = np.vstack((subject, [s]))

        trial = trial + 1

            #ASISXnovo = np.vstack((ASISXnovo,(ASISX[-1,:])[np.logical_not(np.isnan(ASISX[-1,:]))]))
            #ASISYnovo = np.vstack((ASISYnovo,(ASISY[-1,:])[np.logical_not(np.isnan(ASISY[-1,:]))]))
            #ASISZnovo = np.vstack((ASISZnovo,(ASISZ[-1,:])[np.logical_not(np.isnan(ASISZ[-1,:]))]))


            #if (ASISXnovo.size)!=0:
       

filename = 'COPX.pkl'

file = open(filename, 'wb')

# dump information to that file
         
pickle.dump(COPXreal, file)
pickle.dump(COPZreal,file)
pickle.dump(COPX,file)
pickle.dump(COPZ,file)
pickle.dump(d1x,file)
pickle.dump(d1z,file)
pickle.dump(d2x,file)
pickle.dump(d2z,file)
pickle.dump(matrizangx,file)
pickle.dump(matrizangz,file)
pickle.dump(ASISX,file)
pickle.dump(ASISY,file)
pickle.dump(ASISZ,file)
pickle.dump(angulox,file)
pickle.dump(anguloz,file)
pickle.dump(d,file)
pickle.dump(alpha,file)
pickle.dump(subject,file)
pickle.dump(ei,file)
pickle.dump(sinx,file)
pickle.dump(sinz,file)
pickle.dump(cosx,file)
pickle.dump(cosz,file)
pickle.dump(tanx,file)
pickle.dump(tanz,file)

filename = 'COPZ.pkl'
file = open(filename, 'wb')

pickle.dump(COPZreal, file)
print(asisx.shape)
print(ASISX.shape)

<ipython-input-73-a0a5ca5da4aa>:69: RuntimeWarning: invalid value encountered in true_divide
  angulox[trial,:] = np.arcsin(ASISX[trial,:]/d[trial,:])
<ipython-input-73-a0a5ca5da4aa>:70: RuntimeWarning: invalid value encountered in true_divide
  anguloz[trial,:] = np.arcsin(ASISZ[trial,:]/d[trial,:])
<ipython-input-73-a0a5ca5da4aa>:69: RuntimeWarning: invalid value encountered in true_divide
  angulox[trial,:] = np.arcsin(ASISX[trial,:]/d[trial,:])
<ipython-input-73-a0a5ca5da4aa>:70: RuntimeWarning: invalid value encountered in true_divide
  anguloz[trial,:] = np.arcsin(ASISZ[trial,:]/d[trial,:])


(5970,)
(588, 5970)


In [74]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
fs = 100 # Hz
import pickle 
#file = open('important', 'wb')

COPX = np.zeros((49*12, 60*fs-30))
COPZ = np.zeros((49*12, 60*fs-30))

COPXreal = np.zeros((49*12, 60*fs-30))
COPYreal = np.zeros((49*12, 60*fs-30))
COPZreal = np.zeros((49*12, 60*fs-30))

d1x = np.zeros((49*12, 60*fs-30))
d2x = np.zeros((49*12, 60*fs-30))

d1z = np.zeros((49*12, 60*fs-30))
d2z = np.zeros((49*12, 60*fs-30))

matrizangx = np.zeros((49*12, 60*fs-30))
matrizangz = np.zeros((49*12, 60*fs-30))

alpha = np.zeros((49*12, 60*fs-30))


COGX = np.empty((49*12, 60*fs-30))
COGY = np.empty((49*12, 60*fs-30))
COGZ = np.empty((49*12, 60*fs-30))

d = np.zeros((49*12, 60*fs-30))


angulox = np.zeros((49*12, 60*fs-30))
anguloz= np.zeros((49*12, 60*fs-30))

matrizangx = np.zeros((49*12, 60*fs-30))
matrizangz = np.zeros((49*12, 60*fs-30))



subject = np.zeros((0,1))
j=0


trial = 0
for s in range (1,50):
    
    #a = PDSinfo[PDSinfo['Subject']==s].index.tolist()[2]
    
    ei = PDSinfo[PDSinfo['Subject']==s].index.tolist()
    
    
    for i in range(0,len(ei)):
        a = ei[i]
        fname_grf = os.path.join(path2, PDSinfo.Trial[a] + 'grf' + '.txt') 
        fname_mkr = os.path.join(path2, PDSinfo.Trial[a] + 'mkr' + '.txt')
        grf = pd.read_csv(fname_grf, delimiter='\t', header=0, engine='c')
        mkr = pd.read_csv(fname_mkr, delimiter='\t', header=0, engine='c')
        
        mkr = mkr.replace(np.nan,0)
        grf = grf.replace(np.nan,0)
        
        COGX[trial,:] = (mkr['COG_X'].values[30:])
        COGY[trial,:] = (mkr['COG_Y'].values[30:])
        COGZ[trial,:] = (mkr['COG_Z'].values[30:])
        
        mass=PDSinfo['Mass']
        d[trial,:] = np.sqrt(COGX[trial,:]**2+COGY[trial,:]**2+COGZ[trial,:]**2)
        angulox[trial,:] = np.arcsin(COGX[trial,:]/d[trial,:])
        anguloz[trial,:] = np.arcsin(COGZ[trial,:]/d[trial,:])
        sinx = np.sin(angulox[trial,:])
        sinz = np.sin(anguloz[trial,:])
        cosx = np.cos(angulox[trial,:])
        cosz = np.cos(anguloz[trial,:])
        tanx = np.tan(angulox[trial,:])
        tanz = np.tan(anguloz[trial,:])
        matrizangx[trial,:] = angulox[trial,:]
        matrizangz[trial,:] = anguloz[trial,:]


        Jb=(4/3)*(mass.values[s])*((d[trial,:])**2)
        t=grf['Time']
        dt=t[1]-t[0]


        d1x[trial,:] = np.gradient(matrizangx[trial,:],dt)
        d1z[trial,:] = np.gradient(matrizangz[trial,:],dt)

        d2x[trial,:] = np.gradient(d1x[trial,:],dt)
        d2z[trial,:] = np.gradient(d1z[trial,:],dt)

        COPX[trial,:] = d[trial,:]*sinx + Jb/(mass.values[s]*9.8)*(sinx*d1x[trial,:]**2-cosx*d2x[trial,:]) 
        COPZ[trial,:] = d[trial,:]*sinz + Jb/(mass.values[s]*9.8)*(sinz*d1z[trial,:]**2-cosz*d2z[trial,:])




        COPXreal[trial,:] = grf['COPNET_X'].values[30:]
        COPZreal[trial,:] = grf['COPNET_Z'].values[30:]

        subject = np.vstack((subject, [s]))

        trial = trial + 1

            #ASISXnovo = np.vstack((ASISXnovo,(ASISX[-1,:])[np.logical_not(np.isnan(ASISX[-1,:]))]))
            #ASISYnovo = np.vstack((ASISYnovo,(ASISY[-1,:])[np.logical_not(np.isnan(ASISY[-1,:]))]))
            #ASISZnovo = np.vstack((ASISZnovo,(ASISZ[-1,:])[np.logical_not(np.isnan(ASISZ[-1,:]))]))


            #if (ASISXnovo.size)!=0:
       

filename = 'COPX.pkl'

file = open(filename, 'wb')

# dump information to that file
         
pickle.dump(COPXreal, file)
pickle.dump(COPZreal,file)
pickle.dump(COPX,file)
pickle.dump(COPZ,file)
pickle.dump(d1x,file)
pickle.dump(d1z,file)
pickle.dump(d2x,file)
pickle.dump(d2z,file)
pickle.dump(matrizangx,file)
pickle.dump(matrizangz,file)
pickle.dump(COGX,file)
pickle.dump(COGY,file)
pickle.dump(COGZ,file)
pickle.dump(angulox,file)
pickle.dump(anguloz,file)
pickle.dump(d,file)
pickle.dump(alpha,file)
pickle.dump(subject,file)
pickle.dump(ei,file)
pickle.dump(sinx,file)
pickle.dump(sinz,file)
pickle.dump(cosx,file)
pickle.dump(cosz,file)
pickle.dump(tanx,file)
pickle.dump(tanz,file)

filename = 'COPZ.pkl'
file = open(filename, 'wb')

pickle.dump(COPZreal, file)
print(np.isnan(cogx).any())



<ipython-input-74-01c1eae6216e>:69: RuntimeWarning: invalid value encountered in true_divide
  angulox[trial,:] = np.arcsin(COGX[trial,:]/d[trial,:])
<ipython-input-74-01c1eae6216e>:70: RuntimeWarning: invalid value encountered in true_divide
  anguloz[trial,:] = np.arcsin(COGZ[trial,:]/d[trial,:])
<ipython-input-74-01c1eae6216e>:69: RuntimeWarning: invalid value encountered in true_divide
  angulox[trial,:] = np.arcsin(COGX[trial,:]/d[trial,:])
<ipython-input-74-01c1eae6216e>:70: RuntimeWarning: invalid value encountered in true_divide
  anguloz[trial,:] = np.arcsin(COGZ[trial,:]/d[trial,:])
<ipython-input-74-01c1eae6216e>:69: RuntimeWarning: invalid value encountered in true_divide
  angulox[trial,:] = np.arcsin(COGX[trial,:]/d[trial,:])
<ipython-input-74-01c1eae6216e>:70: RuntimeWarning: invalid value encountered in true_divide
  anguloz[trial,:] = np.arcsin(COGZ[trial,:]/d[trial,:])
<ipython-input-74-01c1eae6216e>:69: RuntimeWarning: invalid value encountered in true_divide
  angul

False


In [75]:
np.delete(COPXreal,np.where(COPXreal == 0))

np.delete(COPZreal,np.where(COPXreal == 0))

#np.delete(ASISX,np.where(ASISX == np.nan))


array([-0.000594, -0.000535, -0.000462, ...,  0.012296,  0.012536,
        0.012802])

In [77]:
indices = np.arange(1,50).reshape(-1,1)
np.random.shuffle(indices)
train = indices[:40]
test = indices[40:]
asisx = np.zeros((588,5970))

masstrain = mass.values[train]
masstest = mass.values[test]

np.in1d(subject,train)
print(asisx.shape)
print(subject.shape)

ASISXtrain = ASISX[np.in1d(subject,train),]
COGXtrain = COGZ[np.in1d(subject,train),]
COGXtest = COGX[np.in1d(subject,test),]
ASISXtest = ASISX[np.in1d(subject,test),]
COPXtrain = COPX[np.in1d(subject,train),]
COPXtest = COPX[np.in1d(subject,test),]

ASISYtrain = ASISY[np.in1d(subject,train),]
COGYtrain = COGY[np.in1d(subject,train),]
COGYtest = COGY[np.in1d(subject,test),]
ASISYtest = ASISY[np.in1d(subject,test),]

ASISZtrain = ASISZ[np.in1d(subject,train),]
ASISZtest = ASISZ[np.in1d(subject,test),]
COPZtrain = COPZ[np.in1d(subject,train),]
COPZtest = COPZ[np.in1d(subject,test),]
COGZtrain = COGZ[np.in1d(subject,train),]
COGZtest = COGZ[np.in1d(subject,test),]



(588, 5970)
(588, 1)


In [78]:
def newdist(paramt,x,y,z):
    from sklearn.model_selection import train_test_split
    from scipy.optimize import minimize, rosen, rosen_der
    from sklearn.metrics import mean_squared_error
    dest = np.zeros((60*fs-30))
    d = np.zeros((60*fs-30))
    for i in range(0, len(x)):

        delta,mi = paramt

        d[i] = np.mean(np.sqrt(x[i]**2+y[i]**2+z[i]**2))


        dest[i] = delta * d[i] + mi
    
      
    return dest


In [79]:
#for i in range(0,len(COGXtrain)):
        
        #d= np.sqrt(COGXtrain[i]**2+COGYtrain[i]**2+COGZtrain[i]**2)
        
def meansquarerrordist(paramt,d,ASISXtrain,ASISYtrain,ASISZtrain):
    d = np.zeros((60*fs-30))
    from sklearn.metrics import mean_squared_error
    ytrue = np.zeros((60*fs-30))
    ypredicted = np.zeros((60*fs-30))
    for i in range(0,len(ASISXtrain)):
        #print(COGXtrain.shape)
        
        d[i] = np.mean(np.sqrt(COGXtrain[i]**2+COGYtrain[i]**2+COGZtrain[i]**2))
        
        
        ytrue[i] = d[i]
        
        ypredicted = newdist(paramt,ASISXtrain,ASISYtrain,ASISZtrain)
    
    
    
        
    return mean_squared_error(ytrue, ypredicted)


In [80]:
from sklearn.model_selection import train_test_split
#from scipy import optmize
from scipy.optimize import minimize, rosen, rosen_der
first_guess = [1,1]
res = minimize(meansquarerrordist, first_guess,(d,COGXtrain,COGYtrain,COGZtrain))
coeficientes = res.x
delta=coeficientes[0]
mi=coeficientes[1]
print(delta)
print(mi)

0.9999991574027031
-2.09088914947253e-06


In [81]:
treino = newdist(coeficientes,COGXtrain,COGYtrain,COGZtrain)
teste = newdist(coeficientes,COGXtest,COGYtest,COGZtest)

Regressão linear distância

In [ ]:
def centerofpressure(paramt, M,x,y,z):
    from sklearn.model_selection import train_test_split
    from scipy.optimize import minimize, rosen, rosen_der
    from sklearn.metrics import mean_squared_error
    COPXES = np.zeros((1, 60*fs-30))
    COPXESorig =np.zeros((1, 60*fs-30))
    dest =np.zeros((1,60*fs-30))
    trial = 0 
    sennovox=np.zeros((1,60*fs-30))
    cosnovox =np.zeros((1,60*fs-30))
    sennovoz=np.zeros((1,60*fs-30))
    cosnovoz =np.zeros((1,60*fs-30))
    
    for i in range(0,len(x)):
        
        A,alpha,beta, gama= paramt
        
        d[trial,:] = np.mean(np.sqrt(x[trial,:]**2+y[trial,:]**2+z[trial,:]**2))
        
        dest[trial,:] = delta * d[trial,:] + mi
        
        if (any(np.isnan(dest[trial,:]))) != 'True':
            angulox[trial,:] = np.arcsin(np.tanh(x[trial,:]/dest[trial,:]))
            anguloz[trial,:] = np.arcsin(np.tanh(z[trial,:]/dest[trial,:]))
            sinx=np.sin(angulox[trial,:])
            sinz=np.sin(anguloz[trial,:])
            cosx=np.cos(angulox[trial,:])
            cosz=np.cos(anguloz[trial,:])
            tanx=np.tan(angulox[trial,:])
            tanz=np.tan(anguloz[trial,:])
            matrizangx[i,:] = angulox[trial,:]
            matrizangz[i,:] = anguloz[trial,:]







            sennovox[trial,:] = (sinx)*A + ((1-A**2)**1/2)*(cosx)
            

            cosnovox[trial,:] = (cosx)*A - ((1-A**2)**1/2)*(sinx)
            
            sennovoz[trial,:] = (sinz)*A + ((1-A**2)**1/2)*(cosz)
           

            cosnovoz[trial,:] = (cosz)*A - ((1-A**2)**1/2)*(sinz)




            Jb=(4/3)*(M[trial,:])*((dest[trial,:])**2)
            
            t=grf['Time']
            dt=t[1]-t[0]
            
            d1x[trial,:]=np.gradient(matrizangx[trial,:],dt)
            d1z[trial,:]=np.gradient(matrizangz[trial,:],dt)
            
            
            d2x[trial,:]=np.gradient(d1x[trial,:],dt)
            d2z[trial,:]=np.gradient(d1z[trial,:],dt)
            
            if (any(np.isnan(d1x[trial,:]))) != 'True' and (any(np.isnan(d1z[trial,:]))) != 'True' and (any(np.isnan(d2x[trial,:]))) != 'True' and (any(np.isnan(d2z[trial,:]))) != 'True':

                COPX[trial,:] = dest[trial,:]*sennovox[trial,:] + Jb/(M[trial,:]*9.8)*(sennovox[trial,:]*d1x[trial,:]**2-cosnovox[trial,:]*d2x[trial,:])
                COPZ[trial,:] = dest[trial,:]*sennovoz[trial,:] + Jb/(M[trial,:]*9.8)*(sennovoz[trial,:]*d1z[trial,:]**2-cosnovoz[trial,:]*d2z[trial,:])

                COPXES[trial,:]= alpha*(dest[trial,:]*np.tanh(sennovox[trial,:]) + ((Jb/(M[trial,:]*9.8))*np.tanh(sennovox[trial,:])*d1x[trial,:]**2)) + d2x[trial,:]*(-alpha*np.tanh(cosnovox[trial,:])*Jb/(M[trial,:]*9.8)+beta) + gama
                COPXESorig[trial,:]= COPX[trial,:]
                

        
          
    
    return COPXES



In [ ]:
def meansquarerror(paramt,COPXtrain,masstrain,ASISXtrain,ASISYtrain,ASISZtrain):
    
    from sklearn.metrics import mean_squared_error
    
    A,alpha,beta, gama = paramt
        
    ytrue = COPXtrain
    
    ypredicted = centerofpressure(paramt,masstrain,ASISXtrain,ASISYtrain,ASISZtrain)
    
        
    
    
    return mean_squared_error(ytrue, ypredicted)
        


    

In [ ]:
from sklearn.model_selection import train_test_split
#from scipy import optmize
from scipy.optimize import minimize, rosen, rosen_der
first_guess = [1,1,1,1]
res = minimize(meansquarerror, first_guess,(COPXtrain,masstrain,ASISXtrain,ASISYtrain,ASISZtrain))
coeficientes = res.x
A=coeficientes[0]
alpha=coeficientes[1]
beta=coeficientes[2]
gama=coeficientes[3]

Test

In [ ]:

treinando=centerofpressure(coeficientes, masstrain,ASISXtrain,ASISYtrain,ASISZtrain) 

testando=centerofpressure(coeficientes, masstest,ASISXtest,ASISYtest,ASISZtest)


In [ ]:
time = np.arange(0,6000)/100
for i in range(0,len(ASISXtest)):
    plt.figure()
    plt.plot(time[:5970],detrend(testando[i,:]),color='purple',label='COP predicted')
    plt.plot(time[:5970],detrend(COPXreal[i]),color='blue', label='COP real')
    plt.title('COP real X COP calculated for test {:.1f}'.format(i))
    plt.xlabel('Time')
    plt.legend()
    plt.show()

In [ ]:
sennovo = sinx*A + ((1-A**2)**1/2)*cosx
print(np.max(sennovo))
print(np.min(sennovo))

In [ ]:
cosnovo = cosx*A - ((1-A**2)**1/2)*sinx
print(np.max(cosnovo))
print(np.min(cosnovo))